In [ ]:
from google.colab import files
files.upload()

In [2]:
!unzip data.zip

Archive:  data.zip
  inflating: feature.csv             
  inflating: submission.csv          
  inflating: test.csv                
  inflating: train.csv               


In [3]:
!ls

data.zip  feature.csv  sample_data  submission.csv  test.csv  train.csv


In [4]:
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import warnings

In [6]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [9]:
data_dir = Path('.')
sub_dir = Path('./sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
sample_file = data_dir / 'submission.csv'

target_col = 'Survived'
seed = 42

In [10]:
feature_file = data_dir / 'feature.csv'
feat = pd.read_csv(feature_file, index_col=0)
print(feat.shape)
feat.head()

(1309, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Companion
PassengerId,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,2.1102,S,1.0
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,4.2806,C,1.0
3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,2.1889,S,0.0
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,3.9908,S,1.0
5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,2.2028,S,0.0


In [13]:
feat['Sex'].replace({'female':0,'male':1},inplace=True)
feat.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Companion
PassengerId,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,2.1102,S,1.0
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,4.2806,C,1.0
3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,2.1889,S,0.0
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,3.9908,S,1.0
5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,2.2028,S,0.0


In [14]:
feat['Embarked'].value_counts().sort_index()

C    270
Q    123
S    916
Name: Embarked, dtype: int64

In [15]:
feat['EmbarkedC'] = feat['Embarked'].replace({'C':1,'Q':0, 'S':0})
feat.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Companion,EmbarkedC
PassengerId,,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,2.1102,S,1.0,0
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,4.2806,C,1.0,1
3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,2.1889,S,0.0,0
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,3.9908,S,1.0,0
5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,2.2028,S,0.0,0


In [16]:
feat['EmbarkedQ'] = feat['Embarked'].replace({'C':0,'Q':1, 'S':0})
feat['EmbarkedS'] = feat['Embarked'].replace({'C':0,'Q':0, 'S':1})

In [20]:
feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Survived   1309 non-null   float64
 1   Pclass     1309 non-null   int64  
 2   Name       1309 non-null   object 
 3   Sex        1309 non-null   int64  
 4   Age        1309 non-null   float64
 5   SibSp      1309 non-null   int64  
 6   Parch      1309 non-null   int64  
 7   Ticket     1309 non-null   object 
 8   Fare       1309 non-null   float64
 9   Embarked   1309 non-null   object 
 10  Companion  1309 non-null   float64
 11  EmbarkedC  1309 non-null   int64  
 12  EmbarkedQ  1309 non-null   int64  
 13  EmbarkedS  1309 non-null   int64  
dtypes: float64(4), int64(7), object(3)
memory usage: 153.4+ KB


In [22]:
new_feat_file = data_dir / 'feature2.csv'
feat.to_csv(new_feat_file)

In [23]:
!ls

data.zip      feature.csv  sub		   test.csv
feature2.csv  sample_data  submission.csv  train.csv


In [24]:
files.download(new_feat_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>